In [1]:
from neo4j import GraphDatabase
import json

In [ ]:
def process_neo4j_to_indexed_format(query_results):
    """
    Neo4j 쿼리 결과를 처리하여:
    1. 노드들을 0번부터 인덱싱하여 딕셔너리로 저장
    2. sup, r, sub 관계를 [sup_idx, rel_idx, sub_idx] 형태로 변환
    """

    # 노드와 관계 수집
    nodes = {}  # name -> node_info 매핑
    relations = set()  # 고유 관계 타입들
    triples = []  # (sup_name, rel_type, sub_name) 튜플들

    # 쿼리 결과 처리
    for record in query_results:
        sup = record['sup']
        rel = record['r']
        sub = record['sub']

        # 노드 정보 추출
        sup_name = sup.get('name', f"node_{sup.id}")
        sub_name = sub.get('name', f"node_{sub.id}")
        rel_type = rel.type

        # 노드 정보 저장
        nodes[sup_name] = {
            'name': sup_name,
            'embedding': sup.get('embedding', []),
            'createdTimestamp': sup.get('createdTimestamp', 0),
            'oriTopic': sup.get('oriTopic', ''),
            'neo4j_id': sup.id
        }

        nodes[sub_name] = {
            'name': sub_name,
            'embedding': sub.get('embedding', []),
            'createdTimestamp': sub.get('createdTimestamp', 0),
            'oriTopic': sub.get('oriTopic', ''),
            'neo4j_id': sub.id
        }

        # 관계와 트리플 저장
        relations.add(rel_type)
        triples.append((sup_name, rel_type, sub_name))

    # 노드들을 생성시간 순으로 정렬하여 0부터 순서대로 인덱스 부여
    sorted_node_names = sorted(nodes.keys(),
                              key=lambda x: nodes[x]['createdTimestamp'],
                              reverse=True)  # 최신순 (가장 최근 노드가 0번)

    # 노드 이름 -> 사용자 정의 인덱스 매핑 (0부터 시작)
    name_to_idx = {name: idx for idx, name in enumerate(sorted_node_names)}

    # 관계 타입 -> 고정 인덱스 매핑
    rel_to_idx = {
        'isCauseOf': 0,
        'isGeneralOf': 1
    }
    relation_types = ['isCauseOf', 'isGeneralOf']

    # 사용자 정의 인덱싱된 노드 딕셔너리 생성 (0부터 시작하는 인덱스 -> 노드 정보)
    indexed_nodes = {}
    for idx, name in enumerate(sorted_node_names):
        indexed_nodes[idx] = nodes[name].copy()
        indexed_nodes[idx]['custom_index'] = idx  # 사용자 정의 인덱스
        indexed_nodes[idx]['original_neo4j_id'] = nodes[name]['neo4j_id']  # 원본 Neo4j ID는 참조용으로 보관

    # 관계를 인덱스 형태로 변환
    indexed_relations = []
    for sup_name, rel_type, sub_name in triples:
        sup_idx = name_to_idx[sup_name]
        rel_idx = rel_to_idx[rel_type]
        sub_idx = name_to_idx[sub_name]
        indexed_relations.append([sup_idx, rel_idx, sub_idx])

    return {
        'nodes': indexed_nodes,  # {0: {node_info}, 1: {node_info}, ...} - 사용자 정의 0부터 시작
        'relations': indexed_relations,  # [[0, 0, 1], [1, 1, 2], ...] - 사용자 정의 인덱스 사용
        'node_count': len(indexed_nodes),
        'relation_count': len(indexed_relations),
        'relation_types': relation_types,  # ['isCauseOf', 'isGeneralOf']
        'name_to_idx': name_to_idx,  # 노드명 -> 사용자 정의 인덱스 매핑
        'rel_to_idx': rel_to_idx,  # 관계타입 -> 고정 인덱스 매핑 (isCauseOf: 0, isGeneralOf: 1)
        'idx_to_name': {idx: name for name, idx in name_to_idx.items()},
        'idx_to_rel': {idx: rel for rel, idx in rel_to_idx.items()},
        'indexing_order': 'latest_first'  # 인덱싱 순서 설명
    }

In [ ]:
def extract_topic_subgraph_indexed(neo4j_uri="neo4j://regularmark.iptime.org:37687",
                                  username="neo4j",
                                  password="wlrnek123@"):
    """
    Neo4j에서 최신 노드의 oriTopic 기반으로 관계를 추출하고 인덱스 형태로 변환
    """

    query = """
    MATCH (latest:Youtube)
    WITH latest ORDER BY latest.createdTimestamp DESC LIMIT 1
    WITH latest.oriTopic as targetTopic
    MATCH (sup:Youtube)-[r]->(sub:Youtube)
    WHERE sup.oriTopic = targetTopic AND sub.oriTopic = targetTopic
    AND type(r) IN ['isCauseOf', 'isGeneralOf']
    RETURN sup, r, sub
    """

    driver = GraphDatabase.driver(neo4j_uri, auth=(username, password))

    try:
        with driver.session() as session:
            result = list(session.run(query))
            processed_data = process_neo4j_to_indexed_format(result)
            return processed_data
    finally:
        driver.close()

In [ ]:
def print_results(data):
    """
    결과를 보기 좋게 출력
    """
    print("=== 사용자 정의 노드 딕셔너리 (0부터 시작) ===")
    print(f"총 노드 수: {data['node_count']}")
    print(f"인덱싱 순서: {data['indexing_order']} (가장 최근 노드가 0번)")
    for idx, node_info in data['nodes'].items():
        embedding_preview = node_info['embedding'][:3] if len(node_info['embedding']) > 3 else node_info['embedding']
        print(f"[{idx}] {node_info['name']} (Neo4j ID: {node_info['original_neo4j_id']})")
        print(f"    - embedding: {embedding_preview}{'...' if len(node_info['embedding']) > 3 else ''}")
        print(f"    - oriTopic: {node_info['oriTopic']}")
        print(f"    - createdTimestamp: {node_info['createdTimestamp']}")
        print()

    print("=== 고정 관계 타입 매핑 ===")
    print("isCauseOf -> 0")
    print("isGeneralOf -> 1")
    print(f"실제 사용된 관계 타입: {list(set([data['idx_to_rel'][rel[1]] for rel in data['relations']]))}")
    print()

    print("=== 사용자 정의 인덱스 관계 배열 ===")
    print(f"총 관계 수: {data['relation_count']}")
    print("형태: [sup_idx, rel_idx, sub_idx] - 모든 인덱스는 0부터 시작하는 사용자 정의")
    for i, relation in enumerate(data['relations']):
        sup_idx, rel_idx, sub_idx = relation
        sup_name = data['nodes'][sup_idx]['name']
        rel_name = data['idx_to_rel'][rel_idx]
        sub_name = data['nodes'][sub_idx]['name']
        print(f"{relation} -> {sup_name} --{rel_name}--> {sub_name}")

In [12]:
# 임베딩 배열만 따로 추출하는 유틸리티 함수
def extract_embeddings_array(data):
    """
    인덱스 순서대로 임베딩 배열을 추출
    """
    embeddings = []
    for idx in range(data['node_count']):
        embeddings.append(data['nodes'][idx]['embedding'])
    return embeddings

def extract_relations_for_ml(data):
    """
    머신러닝에서 사용하기 쉬운 형태로 관계 데이터 추출
    """
    return {
        'edge_index': [[rel[0], rel[2]] for rel in data['relations']],  # [source, target] 쌍들
        'edge_type': [rel[1] for rel in data['relations']],  # 관계 타입 인덱스들
        'node_features': extract_embeddings_array(data),  # 노드 임베딩들
        'num_nodes': data['node_count'],
        'num_relations': len(data['relation_types'])
    }

In [ ]:
result = extract_topic_subgraph_indexed()

print("\n=== 사용자 정의 인덱스 vs Neo4j ID 매핑 ===")
for idx in range(result['node_count']):
    node_info = result['nodes'][idx]
    print(f"사용자 정의 인덱스 {idx} -> {node_info['name']} (Neo4j ID: {node_info['original_neo4j_id']})")


=== 사용자 정의 인덱스 vs Neo4j ID 매핑 ===
사용자 정의 인덱스 0 -> 그러기 때문에 (Neo4j ID: 7122)
사용자 정의 인덱스 1 -> 국가적 차원 조치 (Neo4j ID: 7123)
사용자 정의 인덱스 2 -> 예산 확보 (Neo4j ID: 7124)
사용자 정의 인덱스 3 -> 인력 투입 (Neo4j ID: 7125)
사용자 정의 인덱스 4 -> 국내 사이버 보안 체계 점검 (Neo4j ID: 7126)
사용자 정의 인덱스 5 -> 정부 AR 산업 100조원 투입 (Neo4j ID: 7127)
사용자 정의 인덱스 6 -> 보안 체계 구축 정부 기업 소극적 (Neo4j ID: 7128)
사용자 정의 인덱스 7 -> 2023년 국내 기업 정보보호 투자액 평균 29억 원 (Neo4j ID: 7129)
사용자 정의 인덱스 8 -> SKT 약 3% (Neo4j ID: 7130)
사용자 정의 인덱스 9 -> 보안 투자 SKT (Neo4j ID: 7188)
사용자 정의 인덱스 10 -> 대규모 해킹 피해 입음 (Neo4j ID: 7189)
사용자 정의 인덱스 11 -> 전반적 보안 인프라 취약성 알림 (Neo4j ID: 7190)
사용자 정의 인덱스 12 -> 전문가들 AI 확산 사이버 위협 증가 (Neo4j ID: 7191)
사용자 정의 인덱스 13 -> 보안 예산 증액 필요 (Neo4j ID: 7192)
사용자 정의 인덱스 14 -> AI 고도화 진행 (Neo4j ID: 7193)
사용자 정의 인덱스 15 -> 개인 정보 수입사 발생 (Neo4j ID: 7194)
사용자 정의 인덱스 16 -> 안경한 사례 계속 발표 (Neo4j ID: 7195)
사용자 정의 인덱스 17 -> 세정보 출범 (Neo4j ID: 7196)
사용자 정의 인덱스 18 -> 보안 예산 확대 (Neo4j ID: 7197)
사용자 정의 인덱스 19 -> 국가 기관 신설 조언 (Neo4j ID: 7198)


/tmp/ipykernel_1338442/1428551815.py:20: DeprecationWarning: `id` is deprecated, use `element_id` instead
  sup_name = sup.get('name', f"node_{sup.id}")
/tmp/ipykernel_1338442/1428551815.py:21: DeprecationWarning: `id` is deprecated, use `element_id` instead
  sub_name = sub.get('name', f"node_{sub.id}")
/tmp/ipykernel_1338442/1428551815.py:30: DeprecationWarning: `id` is deprecated, use `element_id` instead
  'neo4j_id': sup.id
/tmp/ipykernel_1338442/1428551815.py:38: DeprecationWarning: `id` is deprecated, use `element_id` instead
  'neo4j_id': sub.id


In [17]:
print_results(result)

=== 사용자 정의 노드 딕셔너리 (0부터 시작) ===
총 노드 수: 20
인덱싱 순서: latest_first (가장 최근 노드가 0번)
[0] 그러기 때문에 (Neo4j ID: 7122)
    - embedding: [-0.6235675811767578, 0.5532888174057007, -0.052603889256715775]...
    - oriTopic: ['사고', '정보', '발생', '조사', '문제']
    - createdTimestamp: 1749964114739

[1] 국가적 차원 조치 (Neo4j ID: 7123)
    - embedding: [0.33304497599601746, -0.17304277420043945, -1.560165524482727]...
    - oriTopic: ['사고', '정보', '발생', '조사', '문제']
    - createdTimestamp: 1749964114739

[2] 예산 확보 (Neo4j ID: 7124)
    - embedding: [0.31200775504112244, 0.29941901564598083, -2.261016368865967]...
    - oriTopic: ['사고', '정보', '발생', '조사', '문제']
    - createdTimestamp: 1749964114739

[3] 인력 투입 (Neo4j ID: 7125)
    - embedding: [-0.10386063903570175, 0.680935263633728, -1.2900769710540771]...
    - oriTopic: ['사고', '정보', '발생', '조사', '문제']
    - createdTimestamp: 1749964114739

[4] 국내 사이버 보안 체계 점검 (Neo4j ID: 7126)
    - embedding: [0.410958468914032, -1.3033828735351562, -1.3168995380401611]...
    - ori